In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree

EARTH_RADIUS_M = 6371000

# -----------------------------
# Configuration
# -----------------------------
CROSSWALK_CSV = '../cw.csv'
ACCIDENT_CSV  = '../acc_hotspot.csv'

RADIUS_M = 500  # meters
OUT_CW_ACC_MAP = 'cw_acc_map_500m.csv'          # (NEW) 매핑 테이블

# -----------------------------
# Load
# -----------------------------
cw = pd.read_csv(CROSSWALK_CSV)
acc = pd.read_csv(ACCIDENT_CSV)

# 좌표 컬럼명은 네 데이터에 맞게 유지(노트북 기존 로직 그대로 쓰면 됨)
cw_valid = cw.dropna(subset=['crosswalk_lat','crosswalk_lon']).copy()
acc_valid = acc.dropna(subset=['accident_lat','accident_lon']).copy()

# radians (lat, lon)
cw_rad  = np.radians(cw_valid[['crosswalk_lat','crosswalk_lon']].to_numpy())
acc_rad = np.radians(acc_valid[['accident_lat','accident_lon']].to_numpy())

# BallTree (haversine)
tree = BallTree(acc_rad, metric='haversine')
radius_rad = RADIUS_M / EARTH_RADIUS_M

# -----------------------------
# (A) 500m 내 "모든 사고" 매핑 테이블 만들기  ✅ 핵심
# -----------------------------
inds, dists = tree.query_radius(cw_rad, r=radius_rad, return_distance=True, sort_results=True)

cw_idx = np.repeat(np.arange(len(cw_valid)), [len(x) for x in inds])
acc_idx = np.concatenate(inds) if len(cw_idx) else np.array([], dtype=int)
dist_m  = (np.concatenate(dists) * EARTH_RADIUS_M) if len(cw_idx) else np.array([], dtype=float)

map_df = pd.DataFrame({
    'cw_idx': cw_idx,
    'acc_idx': acc_idx,
    'distance_m': dist_m
})

# 원본 컬럼 붙이기 (prefix로 충돌 방지)
cw_pref  = cw_valid.reset_index(drop=True).add_prefix('cw_')
acc_pref = acc_valid.reset_index(drop=True).add_prefix('acc_')

cw_acc_map = (
    map_df
    .merge(cw_pref, left_on='cw_idx', right_index=True, how='left')
    .merge(acc_pref, left_on='acc_idx', right_index=True, how='left')
    .drop(columns=['cw_idx','acc_idx'])
)

cw_acc_map.to_csv(OUT_CW_ACC_MAP, index=False, encoding='utf-8-sig')
print("saved:", OUT_CW_ACC_MAP, "rows:", len(cw_acc_map))



saved: cw_acc_map_500m.csv rows: 964240


KeyboardInterrupt: 